In [54]:
import win32com.client
import pandas as pd
import datetime

In [55]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

In [56]:
inbox = outlook.GetDefaultFolder(6) #6 is the inbox, can change it for other folders

In [57]:
#see all the folders that are available
for folder in outlook.Folders: 
    print(folder.Name)

Public Folders - isabel.fernandez@maastrichtuniversity.nl
assistant-iberialatam@maastrichtuniversity.nl
Public Folders - info-iberialatam@maastrichtuniversity.nl
Public Folders - assistant-iberialatam@maastrichtuniversity.nl
info-iberialatam@maastrichtuniversity.nl
Outlook Data File
i.fernandezpereira@student.maastrichtuniversity.nl
Public Folders - i.fernandezpereira@student.maastrichtuniversity.nl
isabel.fernandez@maastrichtuniversity.nl


In [58]:
if 1==0: #make this true when you want the main inbox
    messages = inbox.Items
    
else: #use this when trying to access another folder or a delegated mailbox
    
    root_folder = outlook.Folders.Item(2) #this allows you to access different inboxes 
    #make sure correct inbox is being used
    print(root_folder)
    subfolder = root_folder.Folders['Inbox']
    #get emails from that inbox
    messages = subfolder.Items
    
#sort messages by the time they were received
messages.Sort("ReceivedTime")
message = messages.GetFirst()
print(message)

#attach = message.Attachments
#print (body_content)
#print(datetime)
#print(sender)
#print(subject)

assistant-iberialatam@maastrichtuniversity.nl
RE: International College San Francisco de Paula, Seville


In [59]:
#create empty dataframe, fill it in as emails are read
df= pd.DataFrame(columns=['subject', 'sender', 'date', 'content of message'])

In [60]:
#first cast the datetime to string, so that it can be stripped of the timezone and used more easily
#The error correction can propably be improved
def get_date (date, i):
    date= str(date)
    try:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M:%S')
    except ValueError:
        date = datetime.datetime.strptime(date.rstrip("+00:00"), '%Y-%m-%d %H:%M')
    df.loc[i,'date'] =date
#read body firt, remove unwanted text, then feed it to the df
def get_content (body, i):
    body= body.replace("\r"," ") 
    body= body.replace("\n","") 
    body= body.replace("\t","") 
    df.loc[i,'content of message'] = body

In [61]:
for i in range(0,50):
    try:
        df.loc[i,'subject']=message.Subject
        df.loc[i,'sender']=message.Sender
        get_date(message.CreationTime, i)      
        get_content(message.body, i)
        
    except AttributeError as e:
        if e == 'GetNext.Subject':
            df.loc[i,'subject']= None
            df.loc[i,'sender'] = message.Sender
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e == 'GetNext.Sender':
            df.loc[i,'sender'] = None
            get_date(message.CreationTime, i)
            get_content(message.body, i)
        elif e=='GetNext.message.body':
            df.loc[i,'content of message'] = None
           
            
    message= messages.GetNext()



In [62]:
display(df)

,subject,sender,date,content of message
0,RE: International College San Francisco de Pau...,"Boer, Zoe den (BU)",2017-10-30 15:15:51,"Dear César, Thank you very much for visiti..."
1,RE: International College San Francisco de Pau...,César Prado Fernández,2017-10-30 15:48:04,Thanks so much for all your help. Best regar...
2,FW: Contact number #78141: Applying Information,talk2students@maastrichtuniversity.nl,2017-10-30 16:48:28,L.S. A question has been send to our ticket m...
3,Re: Applying Information,Juan Fernando Marin Solana,2017-10-30 17:51:21,"Hi Konrad, Thank You forma your answer. I'm ..."
4,Re: Applying Information,Juan Fernando Marin Solana,2017-10-30 18:23:23,"Wonderful, how can I apply for the loan? It wo..."
5,Re: Applying Information,Juan Fernando Marin Solana,2017-10-30 18:38:17,"Yes please, give me his contact information. I..."
6,"Introducing student employees: Gaba, Noa and D...","Hofman, Patricia (BU)",2017-10-31 14:12:55,My name is Gabriele Gotkovic but you can ca...
7,"Introducing student employees: Gaba, Noa and D...","Hofman, Patricia (BU)",2017-10-31 14:13:06,My name is Gabriele Gotkovic but you can ca...
8,RE: Visit Monday from Spain,Info-college (UCM),2017-11-01 10:36:18,"Dear Konrad, Regarding the questions asked..."
9,Fw: REMINDER NEW LIST!,/o=UNIMAAS/ou=UNIMAAS/cn=Recipients/cn=Aylin.S...,2017-11-01 12:00:01,Aylin Saltik Junior Recruitment Officer/...


In [10]:
#change file name accordingly 
df.to_csv('.\dataset\Delegated_Email.csv')

In [167]:
from Parser import read_email, corpus2sentences, generate_text
from spacy.lang.en import English
def num_question(email):
    counter=0
    for char in email:
        if char == '?':
            counter+=1
    
    return counter

#TODO: create a function to extract questions
#def get_questions(email):
    

#TODO: make it all lowercase
#super basic function, more research needed
def remove_signature(email):
    sep= ''
    #create list of words that show end of email
    end_keywords= ['regards,', 'best,' ]
    for word in email.split():
        if word in end_keywords:
            sep = word
            #remove everything after end keyword
            email = email.split(sep, 1)[0]
    return email
        
def remove_email(email):
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', email)
   # emails.append(re.findall())
    for word in email.split():
        if word in emails:
            email=email.replace(word, '. ')
        
    return email

    
        

In [168]:
string =df.loc[19,'content of message']
string= string.lower()
print(string)

can we make it 14:30? have a meeting with aisha at 14:00       zoé den boer senior marketing & student recruitment advisor, international marketing & communications zoe.denboer@maastrichtuniversity.nl <mailto:zoe.denboer@maastrichtuniversity.nl>  www.maastrichtuniversity.nl <http://www.maastrichtuniversity.nl/>   minderbroedersberg 4-6, 6211 lk maastricht  p.o. box 616, 6200 md maastricht, the netherlands  t 043 3885211 f 043 3885225  please consider your environmental responsibility before printing this e-mail.      from: assistant-iberialatam (bu)  sent: maandag 6 november 2017 12:14 to: boer zoe (den) (bu) subject: re: fiep lisbon review - today     perfect!     konrad eickhoff student assistant for iberia and lati​n america marketing & communications assistant-iberialatam@maastrichtuniversity.nl  <mailto:zoe.denboer@maastrichtuniversity.nl>  www.maastrichtuniversity.nl  <http://www.maastrichtuniversity.nl/>   minderbroedersberg 4-6, 6211 lk maastricht  p.o. box 616, 6200 md maastri

In [169]:
num_question(string)

2

In [170]:
string =remove_signature(string)
print(string)

can we make it 14:30? have a meeting with aisha at 14:00       zoé den boer senior marketing & student recruitment advisor, international marketing & communications zoe.denboer@maastrichtuniversity.nl <mailto:zoe.denboer@maastrichtuniversity.nl>  www.maastrichtuniversity.nl <http://www.maastrichtuniversity.nl/>   minderbroedersberg 4-6, 6211 lk maastricht  p.o. box 616, 6200 md maastricht, the netherlands  t 043 3885211 f 043 3885225  please consider your environmental responsibility before printing this e-mail.      from: assistant-iberialatam (bu)  sent: maandag 6 november 2017 12:14 to: boer zoe (den) (bu) subject: re: fiep lisbon review - today     perfect!     konrad eickhoff student assistant for iberia and lati​n america marketing & communications assistant-iberialatam@maastrichtuniversity.nl  <mailto:zoe.denboer@maastrichtuniversity.nl>  www.maastrichtuniversity.nl  <http://www.maastrichtuniversity.nl/>   minderbroedersberg 4-6, 6211 lk maastricht  p.o. box 616, 6200 md maastri

In [171]:
#string= 'Hello world! bel.f.pereira@gmail.com'
string =remove_email(string)
print(string)

can we make it 14:30? have a meeting with aisha at 14:00       zoé den boer senior marketing & student recruitment advisor, international marketing & communications .  <mailto:. >  www.maastrichtuniversity.nl <http://www.maastrichtuniversity.nl/>   minderbroedersberg 4-6, 6211 lk maastricht  p.o. box 616, 6200 md maastricht, the netherlands  t 043 3885211 f 043 3885225  please consider your environmental responsibility before printing this e-mail.      from: assistant-iberialatam (bu)  sent: maandag 6 november 2017 12:14 to: boer zoe (den) (bu) subject: re: fiep lisbon review - today     perfect!     konrad eickhoff student assistant for iberia and lati​n america marketing & communications .   <mailto:. >  www.maastrichtuniversity.nl  <http://www.maastrichtuniversity.nl/>   minderbroedersberg 4-6, 6211 lk maastricht  p.o. box 616, 6200 md maastricht, the netherlands  t 043 3885211 f 043 3885225  ________________________________  from: boer zoe (den) (bu) sent: 06 november 2017 12:02 to

In [173]:
#create Bag of Words
import numpy as np
import re


In [174]:
def extract_words(text):
    ignore_words=['and', 'a', 'the', 'regards']
    words = re.sub("[^\w]", " ",  text).split()
    words_cleaned = [w.lower() for w in words if w not in ignore_words]
    return words_cleaned 

def tokenize_sentences(documents):
    words = []
    for text in documents:
        w = extract_words(text)
        words.extend(w)
        
    words = sorted(list(set(words)))
    return words

#input: collection of documents and sentece to be vectorized
def bag_of_words(documents, sentence):
    vocab = tokenize_sentences(documents) #get vocabulary of all documents
    sentence_words = extract_words(sentence) #get words on specific sentence
    bag = np.zeros(len(vocab))
    for sent_word in sentence_words:
        for i,word in enumerate(vocab):
            if sent_word == word:
                bag[i] += 1
                
    return np.array(bag)


    

In [175]:
se ='Hello, world! How are you? This is Isabel speaking, hello'
sa = 'Hello! This is another string, which I am using for testing purposes. Hello world'

test = se, sa

In [176]:
bag_of_words(test, sa)

array([1., 1., 0., 1., 2., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
       0.])